In [1]:
from numpy.random import random

# Basic Classes

In [3]:
class BasicCell(object):
    """ The Cell class. """
    
    def __init__(self, size, age):
        self.size = size
        self.age = age

In [4]:
c = BasicCell(size=4, age=10)

In [5]:
c.age

10

In [6]:
class CellPopulation(object):
    """ Population of cells. """
    
    def __init__(self):
        self.population = []
        
    def add_cell(self, cell):
        self.population.append(cell)
        
    def remove_cell(self, cell):
        self.population.remove(cell)

# Type checking & Subclassing

In [11]:
class NotACellError(Exception):
    pass
    
class CellPopulation(object):
    """ Population of cells. """
    
    def __init__(self):
        self.population = []
        
    def add_cell(self, cell):            
        
        if isinstance(cell, BasicCell):
            self.population.append(cell)
        else:
            raise NotACellError("{} is not a Cell".format(cell))
        
    def remove_cell(self, cell):
        self.population.remove(cell)

In [21]:
pop = CellPopulation()
pop.add_cell(BasicCell(4,5))
pop.population

# Cell Magics Methods and Properties

In [36]:
class BetterCell(BasicCell):
    """ The BetterCell class. 
    Contains more methods than BasicCell. """
        
    def grow(self):
        """Grows the cell by a random amount, and ages it by 1. """
        self.size += random()
        self.age += 1
        
    def __repr__(self):
        """ Object representation for identification. """
        return "<Cell Size:{}, Age:{}>".format(self.size, self.age)
    
    def __str__(self):
        """ Human readable string representation. """
        return """==============
|    Cell    |
|  Age: {}   |
|  Size: {:.2f}  |
==============""".format(int(self.age), self.size)
    
    @property
    def size_category(self):
        if self.size > 40:
            return 'Large'
        else:
            return 'Small'   

In [37]:
c = BetterCell(40, 20)

In [38]:
c

<Cell Size:40, Age:20>

In [39]:
print(c)

|    Cell    |
|  Age: 20   |
|  Size: 40.00  |


In [29]:
c.size_category

'Small'

In [35]:
c.grow()
c.size

41.62846566875736

In [41]:
isinstance(c, BasicCell)

True

# Comparison Methods + Addition

In [54]:
class EvenBetterCell(BetterCell):
    """ The EvenBetterCell class. 
    Contains more methods than BetterCell. """
            
    def __repr__(self):
        return self.__str__()
    
    def __lt__(self, other):
        if isinstance(other, BasicCell):
            return self.size < other.size
        else:
            raise NotACellError("Cannot compare Cells with Non-Cells")
        
    def __eq__(self, other):
        if isinstance(other, BasicCell):
            return self.size == other.size
        else:
            raise NotACellError("Cannot compare Cells with Non-Cells")
            
    def __le__(self, other):
        return self.__eq__() or self.__lt__()
    
    def __add__(self, other):
        """ Merges two cells together. """
        if isinstance(other, BasicCell):
            return self.__class__(size = self.size + other.size, 0.5*(self.age + other.age))
        else:
            raise NotACellError("Cannot compare Cells with Non-Cells")
            
    def __truediv__(self, other): # __div__ in Python2
        """ Splits a cell into 'other' peices. """
        if isinstance(other, int):
            half = self.__class__(self.size/other, self.age)
            return (half, half)
        else:
            raise NotImplementedError

In [55]:
c1, c2 = EvenBetterCell(size=10, age=20), EvenBetterCell(size=15, age=30)

In [46]:
c1, c2

(==============
 |    Cell    |
 |  Age: 20   |
 |  Size: 10.00  |
 ==============, ==============
 |    Cell    |
 |  Age: 30   |
 |  Size: 15.00  |
 ==============)

In [47]:
c1 < c2

True

In [52]:
c1 + c2

|    Cell    |
|  Age: 25   |
|  Size: 25.00  |

In [57]:
c2 / 2

TypeError: unsupported operand type(s) for *: 'EvenBetterCell' and 'int'

# Iterators

In [58]:
class BetterCellPopulation(CellPopulation):
    
#    def __init__(self):
#        super().__init__() # Different in Python2
#        self.index = 0

    def __getitem__(self, ix):
        return self.population[ix]
    
#     def __iter__(self):
#         return self

#     def __next__(self):
#         if self.index == len(self.population):
#             self.index = 0
#             raise StopIteration
#         self.index += 1
#         return self.population[self.index]

In [59]:
culture = BetterCellPopulation()
culture.population = [EvenBetterCell(x,y) for x,y in zip(20*random(10), 30*random(10))]

In [60]:
for cell in culture:
    print(cell)

|    Cell    |
|  Age: 27   |
|  Size: 4.81  |
|    Cell    |
|  Age: 0   |
|  Size: 2.05  |
|    Cell    |
|  Age: 4   |
|  Size: 12.10  |
|    Cell    |
|  Age: 6   |
|  Size: 10.30  |
|    Cell    |
|  Age: 11   |
|  Size: 10.58  |
|    Cell    |
|  Age: 15   |
|  Size: 14.93  |
|    Cell    |
|  Age: 11   |
|  Size: 18.89  |
|    Cell    |
|  Age: 8   |
|  Size: 18.85  |
|    Cell    |
|  Age: 5   |
|  Size: 17.75  |
|    Cell    |
|  Age: 6   |
|  Size: 9.15  |


# 

In [62]:
import time

class SimulationTimer(object):
    
    def __init__(self):
        pass
    
    def __enter__(self):
        self.culture = BetterCellPopulation()
        self.time_start = time.time()
        return self.culture
    
    def __exit__(self, *args):
        self.time_end = time.time()
        print("Simulation Complete: Took {}s".format(self.time_end - self.time_start))
        print("Final State:")
        for cell in self.culture:
            print(cell)

In [64]:
with SimulationTimer() as sim1:
    sim1.add_cell(EvenBetterCell(10, 10))
    sim1.add_cell(EvenBetterCell(20, 30))
    time.sleep(5)
    for cell in sim1:
        cell.grow()

Simulation Complete: Took 5.005125045776367s
Final State:
|    Cell    |
|  Age: 11   |
|  Size: 10.50  |
|    Cell    |
|  Age: 31   |
|  Size: 20.07  |


# Other Magics


__call__

__contains__

In [ ]:
__contains__

__int__

__hash__

